# Import thư viện.

In [ ]:
# Nguyen Truong Huy.
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, count, when, to_timestamp, split, regexp_replace, row_number, sum, count_distinct
from functools import reduce
import pyspark.pandas as ps

# Khởi tạo Spark Session.

In [ ]:
spark = SparkSession.builder \
    .appName("Analysis") \
    .getOrCreate()

# Đọc file dữ liệu và tổng quan.

In [ ]:
category_df = spark.read.csv("./data/preprocessed_data.csv", header=True, inferSchema=True)

In [ ]:
category_df.show()

In [ ]:
category_df.describe().show()

In [ ]:
def dataframe_info(df):
    print(f"{'-'*40}")
    print(f"DataFrame thông tin:")
    print(f"Số dòng: {df.count()}")
    print(f"Số cột: {len(df.columns)}")
    print(f"{'-'*40}")
    print("Schema:")
    df.printSchema()
    print(f"{'-'*40}")
    print("Số giá trị null trong mỗi cột:")
    null_counts = df.select([
        count(when(col(c).isNull(), c)).alias(c) for c in df.columns
    ])
    null_counts.show()

In [ ]:
dataframe_info(category_df)

# Phân tích.

In [ ]:
# Những categories có trong dataset.
category_df.select('category_id').distinct().show()

In [ ]:
# Tổng số categories.
category_df.select('category_id').distinct().count()

## Category nằm top trending nhiều nhất.

In [ ]:
# Đếm và xếp giảm dần theo số video.
category_df.groupBy('category_id').count().orderBy("count", ascending=False).show()

In [ ]:
# Plot.
pandas_category_df = category_df.toPandas()
ax = pandas_category_df['category_id'].value_counts().plot.bar(figsize = (20, 9))

ax.set_xlabel('Categories')
ax.set_ylabel('Number of Videos')

## Category có tổng số lượng views cao nhất (Tổng số view (cao nhất) của các vid thuộc category cụ thể).

#### Tổng số videos.

In [ ]:
# Lọc ra các tên videos do có sự trùng lặp của đặc trưng bộ dữ liệu.
category_df.select('title').distinct().show()

In [ ]:
# Tổng số videos khác nhau.
category_df.select('title').distinct().count()

#### Lọc lấy số view cao nhất của từng video.

In [ ]:
windowSpec = Window.partitionBy("title").orderBy(col("views").desc())
windowSpec

In [ ]:
view_df = category_df.withColumn("view_rank", row_number().over(windowSpec))
view_df.show()

In [ ]:
# Chỉ lấy số views cao nhất của mỗi video.
view_df = view_df.filter(col('view_rank') == 1)
view_df.show()

#### Tính tổng số view theo category.

In [ ]:
# Tạo dataframe với cột category và tổng số views mỗi category.
total_view_df = view_df.groupBy('category_id').agg(sum('views').alias('total_views')).orderBy("total_views", ascending=False)
total_view_df = total_view_df.withColumn("total_views", col("total_views").cast("long"))
total_view_df.show()

#### Plot biểu đồ.

In [ ]:
import matplotlib.ticker as mtick
pandas_total_view_df = total_view_df.toPandas()
pandas_without_music = pandas_total_view_df[pandas_total_view_df['category_id'] != 'Music']

ax = pandas_total_view_df.plot.bar(x='category_id', y='total_views', figsize=(20, 9))
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{x:,.0f}'))

ax.set_xlabel('Categories')
ax.set_ylabel('Total Views')

In [ ]:
# Chủ đề âm nhạc quá trội nên tạo plot khác để thấy các giá trị phía sau rõ hơn. 
ax = pandas_without_music.plot.bar(x='category_id', y='total_views', figsize=(20, 6))
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{x:,.0f}'))  
ax.set_xlabel('Categories')
ax.set_ylabel('Total Views')

## Category có nhiều kênh youtube làm nhất.

In [ ]:
# Đếm tổng số kênh khác nhau theo mỗi chủ đề.
channel_df = category_df.groupBy('category_id').agg(count_distinct('channel_title').alias('total_channels')).orderBy("total_channels", ascending=False)
channel_df.show()

#### Plot biểu đồ.

In [ ]:
pandas_channel_df = channel_df.toPandas()
ax = pandas_channel_df.plot.bar(x='category_id', y='total_channels', figsize=(20, 9))
ax.set_xlabel('Categories')
ax.set_ylabel('Total Channels')